In [ ]:
import cv2
import numpy as np

def gaussianSmoothing(img, sigma):
    ksize = int(np.ceil(6*sigma))
    if ksize % 2 == 0:
        ksize += 1
    kernel = cv2.getGaussianKernel(ksize, sigma)
    smoothedImg = cv2.sepFilter2D(img, -1, kernel, kernel)
    return smoothedImg

def unsharpMasking(img, sigma):
    blurredImg = gaussianSmoothing(img, sigma)
    unsharpImg = cv2.addWeighted(img, 2.0, blurredImg, -1.0, 0)
    return unsharpImg

def combineForegroundBackground(fgImg, fgMask, bgImg, topLeft, bgSigma, fgSigma):
    # smooth image with sigma value
    smoothedBgImg = gaussianSmoothing(bgImg, bgSigma)

    # sharpen image with sigma value
    sharpenedFgImg = unsharpMasking(fgImg, fgSigma)

    fgH, fgW = fgImg.shape[:2]
    bgH, bgW = smoothedBgImg.shape[:2]
    x1, y1 = max(topLeft[1], 0), max(topLeft[0], 0)
    x2, y2 = min(topLeft[1] + fgW, bgW), min(topLeft[0] + fgH, bgH)
    fgStartX, fgStartY = x1 - topLeft[0], y1 - topLeft[1]
    fgEndX, fgEndY = fgStartX + (x2 - x1), fgStartY + (y2 - y1)

    # Create a new background image
    newBgImg = np.copy(smoothedBgImg)

    for y in range(fgStartY, fgEndY):
        for x in range(fgStartX, fgEndX):
            if fgMask[y][x] != 0:
                newBgImg[y + topLeft[0]][x + topLeft[1]] = fgImg[y][x]

    return newBgImg


# Loadimages
fgImg = cv2.imread("foreground.jpg")
bgImg = cv2.imread("background.png")

# Load mask image
fgMask = cv2.imread("mask.png", 0)


# In here we can change the sigma parameters
result = combineForegroundBackground(fgImg, fgMask, bgImg, (0, 0),0,0)

# we display the result
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()
